In [1]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import glob

In [2]:
DATA_DIR = Path('../../results/jatos_results_data_20231027135021')
#DATA_DIR = Path('../../results/jatos_results_data_20231020151141') # caoimhin
assert DATA_DIR.exists()

# use glob to get all files matching *.txt anywhere in the tree under DIRNAME
glob_string = str(DATA_DIR) + r'/**/*.txt'
print(glob_string)
filenames = glob.glob(glob_string, recursive=True)
filenames


../../results/jatos_results_data_20231027135021/**/*.txt


['../../results/jatos_results_data_20231027135021/study_result_317676/comp-result_444707/data.txt',
 '../../results/jatos_results_data_20231027135021/study_result_315567/comp-result_442242/data.txt',
 '../../results/jatos_results_data_20231027135021/study_result_315746/comp-result_442451/data.txt',
 '../../results/jatos_results_data_20231027135021/study_result_315771/comp-result_442482/data.txt',
 '../../results/jatos_results_data_20231027135021/study_result_315772/comp-result_442483/data.txt',
 '../../results/jatos_results_data_20231027135021/study_result_315898/comp-result_442636/data.txt',
 '../../results/jatos_results_data_20231027135021/study_result_315949/comp-result_442691/data.txt',
 '../../results/jatos_results_data_20231027135021/study_result_316005/comp-result_442752/data.txt',
 '../../results/jatos_results_data_20231027135021/study_result_316973/comp-result_443802/data.txt',
 '../../results/jatos_results_data_20231027135021/study_result_316991/comp-result_443820/data.txt',


In [3]:
import re
named_pattern = re.compile(r'^id(?P<scene_id>\d+)_v(?P<version>\d+)_b(?P<blue_count>\d+)_r(?P<red_count>\d+)\..+$')

In [4]:
import json

all_records = []
for fn in filenames:
    p = Path(fn)
    study = p.parts[-3]
    component = p.parts[-2]
    subject_id = component[-6:]
    try:
        with open(fn, 'r', encoding='utf-8') as jf:
            # Load the JSON data and store it in the 'data' dictionary using the file path as the key
            js = json.load(jf)

            record_common = {}

            record_common['browser'] = js['context']['browser']['name']
            record_common['screen_w'] = js['context']['screen']['width']
            record_common['screen_h'] = js['context']['screen']['height']
            record_common['os'] = js['context']['system']['os']
            record_common['exp_start_date'] = js['context']['date']['startdateUTC']
            record_common['exp_start_time'] = js['context']['date']['starttime']

            extra_questions = js['data'][-1]
            record_common['favorite_team'] = extra_questions['favourite_team']
            record_common['favorite_color'] = extra_questions['favourite_color']
            record_common['subject_id'] = subject_id
            # TODO: replace with questiona: 
            # * The usual outfit of any of your favourite teams include (blue/red/both/neither)
            # * Your favourite colours include (blue/red/both/neither)

            _records = []
            for i, datum in enumerate(js['data'][:-1]):  # -1 because the last entry seems to be empty  TODO: check this
                record = {}

                record['age'] = datum['age']
                record['response_side'] = datum['color_response']
                record['impaired_vision'] = datum['daltonic']  # TODO
                record['experience'] = datum['football']
                # datetime = datum['datetime']  # e.g. "Mon Oct 09 2023 15:45:58 GMT+0200 (Central European Summer Time)" or "Mon Oct 09 2023 15:46:03 GMT+0200 (Ora legale dell’Europa centrale)"
                record['gender'] = datum['gender']
                #is_browser = datum['is_browser']  # TODO: maybe needed if people do it on the computer
                #opensesame_codename = datum['opensesame_codename']
                #opensesame_version = datum['opensesame_version']
                record['session_id'] = datum['sessionid']
                record['inner_width'] = datum['innerWidth']
                #record['filename'] = f"id{datum['image_nr']}_v{datum['version']}_r{datum['number_of_red']}_b{datum['number_of_blue']}.jpg"  # TODO
                filename = datum['stimuli_name']
                assert int(datum['cycle_count_var']) == i
                record['trial_no'] = i
                record['filename'] = filename
                pi = re.match(named_pattern, filename).groupdict()
                record['scene_id'] = int(pi['scene_id'])
                record['red_count'] = int(pi['red_count'])
                record['blue_count'] = int(pi['blue_count'])
                record['version'] =  int(pi['version'])
                record['response_time'] = int(datum['reactionTimeAnswer'])

                record['expected_side'] = 'red' if record['red_count'] > record['blue_count'] else 'blue' if record['red_count'] < record['blue_count'] else 'neither'

                _records.append(record | record_common)
            
            all_records = all_records  + _records
    except Exception as e:
        print(f"Error loading JSON data from {fn}: {e}")
        continue

df = pd.DataFrame.from_records(all_records)
df.head()

Error loading JSON data from ../../results/jatos_results_data_20231027135021/study_result_317117/comp-result_443962/data.txt: Expecting value: line 1 column 2 (char 1)
Error loading JSON data from ../../results/jatos_results_data_20231027135021/study_result_317699/comp-result_444733/data.txt: Expecting value: line 1 column 2 (char 1)
Error loading JSON data from ../../results/jatos_results_data_20231027135021/study_result_317699/comp-result_444734/data.txt: Expecting value: line 1 column 2 (char 1)
Error loading JSON data from ../../results/jatos_results_data_20231027135021/study_result_317731/comp-result_444771/data.txt: Expecting value: line 1 column 2 (char 1)


,age,response_side,impaired_vision,experience,gender,session_id,inner_width,trial_no,filename,scene_id,...,expected_side,browser,screen_w,screen_h,os,exp_start_date,exp_start_time,favorite_team,favorite_color,subject_id
0,23,red,no,never,female,1697797568418,1512,0,id17_v1_b6_r5.jpg,17,...,blue,Netscape,1512,982,MacIntel,20-09-23,12:26:00,other,other,444707
1,23,blue,no,never,female,1697797568418,1512,1,id10_v2_b7_r6.jpg,10,...,blue,Netscape,1512,982,MacIntel,20-09-23,12:26:00,other,other,444707
2,23,blue,no,never,female,1697797568418,1512,2,id12_v1_b7_r6.jpg,12,...,blue,Netscape,1512,982,MacIntel,20-09-23,12:26:00,other,other,444707
3,23,blue,no,never,female,1697797568418,1512,3,id20_v1_b7_r6.jpg,20,...,blue,Netscape,1512,982,MacIntel,20-09-23,12:26:00,other,other,444707
4,23,red,no,never,female,1697797568418,1512,4,id5_v2_b7_r6.jpg,5,...,blue,Netscape,1512,982,MacIntel,20-09-23,12:26:00,other,other,444707


In [5]:
df.describe()

,session_id,inner_width,trial_no,scene_id,red_count,blue_count,version,response_time,screen_w,screen_h
count,2.940000e+03,2940.000000,2940.000000,2940.000000,2940.000000,2940.000000,2940.000000,2940.000000,2940.000000,2940.000000
mean,1.697834e+12,1523.668707,29.500000,15.500000,6.216667,6.216667,1.500000,1426.123469,1600.081633,999.530612
std,1.573814e+08,556.616377,17.321048,8.656914,0.754928,0.754928,0.500085,5618.929138,582.146730,217.555766
min,1.697622e+12,360.000000,0.000000,1.000000,5.000000,5.000000,1.000000,178.000000,360.000000,720.000000
25%,1.697742e+12,1280.000000,14.750000,8.000000,6.000000,6.000000,1.000000,698.750000,1368.000000,864.000000
50%,1.697802e+12,1470.000000,29.500000,15.500000,6.000000,6.000000,1.500000,934.000000,1536.000000,956.000000
75%,1.697816e+12,1728.000000,44.250000,23.000000,7.000000,7.000000,2.000000,1383.000000,1920.000000,1080.000000
max,1.698262e+12,3440.000000,59.000000,30.000000,8.000000,8.000000,2.000000,220947.000000,3440.000000,1600.000000


In [6]:
df.to_csv('./data/jatos.csv', index=None)

In [7]:
df

,age,response_side,impaired_vision,experience,gender,session_id,inner_width,trial_no,filename,scene_id,...,expected_side,browser,screen_w,screen_h,os,exp_start_date,exp_start_time,favorite_team,favorite_color,subject_id
0,23,red,no,never,female,1697797568418,1512,0,id17_v1_b6_r5.jpg,17,...,blue,Netscape,1512,982,MacIntel,20-09-23,12:26:00,other,other,444707
1,23,blue,no,never,female,1697797568418,1512,1,id10_v2_b7_r6.jpg,10,...,blue,Netscape,1512,982,MacIntel,20-09-23,12:26:00,other,other,444707
2,23,blue,no,never,female,1697797568418,1512,2,id12_v1_b7_r6.jpg,12,...,blue,Netscape,1512,982,MacIntel,20-09-23,12:26:00,other,other,444707
3,23,blue,no,never,female,1697797568418,1512,3,id20_v1_b7_r6.jpg,20,...,blue,Netscape,1512,982,MacIntel,20-09-23,12:26:00,other,other,444707
4,23,red,no,never,female,1697797568418,1512,4,id5_v2_b7_r6.jpg,5,...,blue,Netscape,1512,982,MacIntel,20-09-23,12:26:00,other,other,444707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2935,26,blue,no,last-month,male,1698262290769,2048,55,id19_v1_b6_r7.jpg,19,...,red,Netscape,2048,1152,Win32,25-09-23,21:29:55,other,blue,450306
2936,26,blue,no,last-month,male,1698262290769,2048,56,id21_v2_b7_r6.jpg,21,...,blue,Netscape,2048,1152,Win32,25-09-23,21:29:55,other,blue,450306
2937,26,blue,no,last-month,male,1698262290769,2048,57,id14_v1_b7_r6.jpg,14,...,blue,Netscape,2048,1152,Win32,25-09-23,21:29:55,other,blue,450306
2938,26,blue,no,last-month,male,1698262290769,2048,58,id30_v1_b7_r6.jpg,30,...,blue,Netscape,2048,1152,Win32,25-09-23,21:29:55,other,blue,450306


In [8]:
df['is_correct'] = df.expected_side == df.response_side
df.groupby('expected_side').is_correct.mean()

expected_side
blue    0.735374
red     0.628571
Name: is_correct, dtype: float64